In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile9 = pd.read_pickle("BBB_Train_Decile9.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile9.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile9, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 9')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 45/100
3087/3087 [==============================] - 14s 4ms/step - loss: 0.3015 - acc: 0.8873 - val_loss: 0.2830 - val_acc: 0.8990
Epoch 46/100
3087/3087 [==============================] - 14s 5ms/step - loss: 0.3090 - acc: 0.8908 - val_loss: 0.2831 - val_acc: 0.8977
Epoch 47/100
3087/3087 [==============================] - 14s 4ms/step - loss: 0.3020 - acc: 0.8886 - val_loss: 0.2883 - val_acc: 0.8964
Epoch 48/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3035 - acc: 0.8889 - val_loss: 0.2824 - val_acc: 0.8990
Epoch 49/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3026 - acc: 0.8921 - val_loss: 0.2810 - val_acc: 0.9016
Epoch 50/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3018 - acc: 0.8860 - val_loss: 0.2817 - val_acc: 0.8990
Epoch 51/100
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2974 - acc: 0.8912 - val_loss: 0.2812 - val_acc: 0.8977
Epoch 52/100
3087/3087 [=================

Epoch 4/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3138 - acc: 0.8905 - val_loss: 0.3130 - val_acc: 0.8860
Epoch 5/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3069 - acc: 0.8908 - val_loss: 0.2896 - val_acc: 0.8938
Epoch 6/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2968 - acc: 0.8912 - val_loss: 0.2869 - val_acc: 0.8873
Epoch 7/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2932 - acc: 0.8921 - val_loss: 0.2747 - val_acc: 0.9003
Epoch 8/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2966 - acc: 0.8895 - val_loss: 0.2761 - val_acc: 0.8977
Epoch 9/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2929 - acc: 0.8892 - val_loss: 0.2756 - val_acc: 0.8977
Epoch 10/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2886 - acc: 0.8947 - val_loss: 0.2719 - val_acc: 0.9003
Epoch 11/150
3087/3087 [=======================

3087/3087 [==============================] - 12s 4ms/step - loss: 0.2700 - acc: 0.8983 - val_loss: 0.2715 - val_acc: 0.8951
Epoch 63/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2701 - acc: 0.9015 - val_loss: 0.2716 - val_acc: 0.8951
Epoch 64/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2700 - acc: 0.8999 - val_loss: 0.2717 - val_acc: 0.8951
Epoch 65/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2705 - acc: 0.9006 - val_loss: 0.2717 - val_acc: 0.8951
Epoch 66/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2685 - acc: 0.8999 - val_loss: 0.2721 - val_acc: 0.8925
Epoch 67/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2694 - acc: 0.8973 - val_loss: 0.2719 - val_acc: 0.8951
Epoch 68/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2687 - acc: 0.9018 - val_loss: 0.2720 - val_acc: 0.8925
Epoch 69/150
3087/3087 [==============================

Epoch 122/150
3087/3087 [==============================] - 10s 3ms/step - loss: 0.2717 - acc: 0.9009 - val_loss: 0.2724 - val_acc: 0.8925
Epoch 123/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2661 - acc: 0.9022 - val_loss: 0.2723 - val_acc: 0.8938
Epoch 124/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2685 - acc: 0.9022 - val_loss: 0.2724 - val_acc: 0.8925
Epoch 125/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2609 - acc: 0.9048 - val_loss: 0.2725 - val_acc: 0.8912
Epoch 126/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2666 - acc: 0.8989 - val_loss: 0.2726 - val_acc: 0.8912
Epoch 127/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2690 - acc: 0.8989 - val_loss: 0.2723 - val_acc: 0.8925
Epoch 128/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2645 - acc: 0.8993 - val_loss: 0.2726 - val_acc: 0.8912
Epoch 129/150
3087/3087 [=========

...

3087/3087 [==============================] - 11s 4ms/step - loss: 0.3281 - acc: 0.8827 - val_loss: 0.3036 - val_acc: 0.8951
Epoch 64/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3212 - acc: 0.8827 - val_loss: 0.3036 - val_acc: 0.8977
Epoch 65/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3238 - acc: 0.8792 - val_loss: 0.3036 - val_acc: 0.8938
Epoch 66/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3250 - acc: 0.8821 - val_loss: 0.3035 - val_acc: 0.8964
Epoch 67/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3217 - acc: 0.8834 - val_loss: 0.3034 - val_acc: 0.8951
Epoch 68/100
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3229 - acc: 0.8788 - val_loss: 0.3034 - val_acc: 0.8964
Epoch 69/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3234 - acc: 0.8860 - val_loss: 0.3033 - val_acc: 0.8964
Epoch 70/100
3087/3087 [==============================

3087/3087 [==============================] - 12s 4ms/step - loss: 0.2770 - acc: 0.8976 - val_loss: 0.2916 - val_acc: 0.8886
Epoch 23/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2837 - acc: 0.8928 - val_loss: 0.2756 - val_acc: 0.8951
Epoch 24/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2790 - acc: 0.8986 - val_loss: 0.2701 - val_acc: 0.9016
Epoch 25/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2768 - acc: 0.8954 - val_loss: 0.2698 - val_acc: 0.9003

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 26/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2665 - acc: 0.8989 - val_loss: 0.2653 - val_acc: 0.8977
Epoch 27/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2723 - acc: 0.8983 - val_loss: 0.2654 - val_acc: 0.9016
Epoch 28/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2677 - acc: 0.8986 - val_los

3087/3087 [==============================] - 12s 4ms/step - loss: 0.2645 - acc: 0.9028 - val_loss: 0.2662 - val_acc: 0.9016
Epoch 81/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2676 - acc: 0.8967 - val_loss: 0.2662 - val_acc: 0.9016
Epoch 82/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2622 - acc: 0.9015 - val_loss: 0.2662 - val_acc: 0.9016
Epoch 83/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2667 - acc: 0.9009 - val_loss: 0.2664 - val_acc: 0.9028
Epoch 84/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2657 - acc: 0.8983 - val_loss: 0.2663 - val_acc: 0.9016
Epoch 85/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2651 - acc: 0.9028 - val_loss: 0.2664 - val_acc: 0.9016
Epoch 86/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2659 - acc: 0.9028 - val_loss: 0.2663 - val_acc: 0.9016
Epoch 87/200
3087/3087 [==============================

3087/3087 [==============================] - 12s 4ms/step - loss: 0.2691 - acc: 0.8996 - val_loss: 0.2668 - val_acc: 0.9016
Epoch 140/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2709 - acc: 0.8983 - val_loss: 0.2668 - val_acc: 0.9016
Epoch 141/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2701 - acc: 0.8976 - val_loss: 0.2668 - val_acc: 0.9016
Epoch 142/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2691 - acc: 0.9006 - val_loss: 0.2668 - val_acc: 0.9016
Epoch 143/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2653 - acc: 0.8983 - val_loss: 0.2670 - val_acc: 0.9016
Epoch 144/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2694 - acc: 0.9022 - val_loss: 0.2670 - val_acc: 0.9016
Epoch 145/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2637 - acc: 0.9006 - val_loss: 0.2669 - val_acc: 0.9016
Epoch 146/200
3087/3087 [=======================

3087/3087 [==============================] - 12s 4ms/step - loss: 0.2693 - acc: 0.9009 - val_loss: 0.2675 - val_acc: 0.9016
Epoch 199/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2667 - acc: 0.9012 - val_loss: 0.2675 - val_acc: 0.9016
Epoch 200/200
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2628 - acc: 0.8999 - val_loss: 0.2674 - val_acc: 0.9016
Test accuracy: 0.9015544041450777
Train on 3087 samples, validate on 772 samples
Epoch 1/50
3087/3087 [==============================] - 14s 5ms/step - loss: 0.6207 - acc: 0.6482 - val_loss: 0.5637 - val_acc: 0.7176
Epoch 2/50
3087/3087 [==============================] - 12s 4ms/step - loss: 0.5335 - acc: 0.7243 - val_loss: 0.4283 - val_acc: 0.8277
Epoch 3/50
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4138 - acc: 0.8571 - val_loss: 0.3544 - val_acc: 0.8899
Epoch 4/50
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3560 - acc: 0.8776 - val_loss: 0.31

In [ ]:
0.8297

0.8297

In [ ]:
#0.92, 0.70 recall

In [ ]:
#0.90, recall = 0.79